# Overview

# Setup
This notebook implements a robust Weather ETL Pipeline designed to process, store, and analyze weather data. The architecture leverages Azure Databricks and a Lakehouse architecture, ensuring scalability, efficiency, and flexibility for future extensions. The pipeline processes weather data from the National Weather Service API, stores it in a Delta table, and answers two specific analytical queries using SQL.

**Compute:** I used Azure Databricks as the processing engine to leverage its distributed computing capabilities and seamless integration with Azure services.

**Storage:** I chose Azure Blob Storage as the data lake to store raw and intermediate data in Parquet format, which ensures efficient storage and querying.

**Data Management:** I utilized Delta Tables in the Unity Catalog for structured, ACID-compliant storage with centralized governance, making it easy to query and maintain high data quality.

## Libraries
The notebook imports the necessary libraries, such as requests for API interaction and PySpark for scalable data processing. These libraries were chosen to ensure efficiency, scalability, and compatibility with the Databricks environment.

In [0]:
import requests
import json
import logging
from typing import Any, Dict, List, Optional
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType
from pyspark.sql.functions import col, to_timestamp, current_timestamp, date_sub

I used a Logger because it provides a robust and structured way to monitor and debug pipelines, ensuring better traceability and maintainability compared to simple print statements.

In [0]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("WeatherDataPipeline")

INFO:py4j.clientserver:Received command c on object id p0


## Constants
Constants are defined for API endpoints, table names, and configurations. Using constants ensures the code is maintainable and easy to adapt to future changes, such as adding new stations or modifying the database structure.

In [0]:
HEADERS = {"User-Agent": "(myweatherapp.com, contact@myweatherapp.com)"}
STATION_ID = "0128W"
BASE_URL = "https://api.weather.gov"
OBSERVATIONS_ENDPOINT = f"{BASE_URL}/stations/{STATION_ID}/observations"
STATION_ENDPOINT = f"{BASE_URL}/stations/{STATION_ID}"
CATALOG_NAME = "company_data"
SCHEMA_NAME = "weather"
TABLE_NAME = "station_observations"
FULL_TABLE_NAME = f"{CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}"

INFO:py4j.clientserver:Received command c on object id p0


## Functions

**Data Fetching:** The fetch_weather_data_last_7_days function retrieves data for the last 7 days. Filtering is handled in Spark, ensuring performance and scalability for large datasets.

**Data Transformation:** The transform_data_to_dataframe function converts raw API data into a structured Spark DataFrame, processing timestamps and rounding numeric values for consistency.

**Data Storage:** The upsert_to_table function implements a MERGE operation to prevent duplicate records during subsequent pipeline runs. This ensures data integrity, especially when re-running the pipeline.

In [0]:
def fetch_station_metadata(station_id: str) -> Optional[Dict[str, Any]]:
    """
    Fetch metadata for a given station, including timezone.
    
    Args:
        station_id (str): The station ID to fetch metadata for.
        
    Returns:
        Optional[Dict[str, Any]]: A dictionary containing metadata or None if unavailable.
    """
    station_url = f"{BASE_URL}/stations/{station_id}"
    try:
        response = requests.get(station_url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            return {
                "station_id": station_id,
                "station_name": data.get("properties", {}).get("name", None),
                "timezone": data.get("properties", {}).get("timeZone", None)
            }
        else:
            logger.warning(f"Failed to fetch metadata for station {station_id}. Status code: {response.status_code}")
            return None
    except Exception as e:
        logger.error(f"Error fetching metadata for station {station_id}: {e}")
        return None

INFO:py4j.clientserver:Received command c on object id p0


In [0]:
def fetch_weather_data(station_id: str) -> List[Dict[str, Any]]:
    """
    Fetch weather observation data for a given station.
    
    Args:
        station_id (str): The station ID to fetch data for.
        
    Returns:
        List[Dict[str, Any]]: A list of observation dictionaries.
    """
    try:
        response = requests.get(OBSERVATIONS_ENDPOINT, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            return data.get("features", [])
        else:
            logger.warning(f"Failed to fetch observations for station {station_id}. Status code: {response.status_code}")
            return []
    except Exception as e:
        logger.error(f"Error fetching observations for station {station_id}: {e}")
        return []

INFO:py4j.clientserver:Received command c on object id p0


In [0]:
def transform_data_to_dataframe(observations: List[Dict[str, Any]], metadata: Dict[str, Any]) -> DataFrame:
    """
    Transform observation data and metadata into a PySpark DataFrame.

    Args:
        observations (List[Dict[str, Any]]): A list of observation dictionaries.
        metadata (Dict[str, Any]): Metadata containing timezone and station details.

    Returns:
        DataFrame: A PySpark DataFrame.
    """
    schema = StructType(
        [
            StructField("station_id", StringType(), True),
            StructField("station_name", StringType(), True),
            StructField("timezone", StringType(), True),
            StructField("latitude", DoubleType(), True),
            StructField("longitude", DoubleType(), True),
            StructField("timestamp_str", StringType(), True),  # Store as String initially
            StructField("temperature", DoubleType(), True),
            StructField("wind_speed", DoubleType(), True),
            StructField("humidity", DoubleType(), True),
        ]
    )

    def safe_round(value: Any, decimals: int = 2) -> Optional[float]:
        return round(float(value), decimals) if value is not None else None

    records = []
    for observation in observations:
        props = observation.get("properties", {})
        coords = observation.get("geometry", {}).get("coordinates", [None, None])
        records.append(
            {
                "station_id": metadata["station_id"],
                "station_name": metadata["station_name"],
                "timezone": metadata["timezone"],
                "latitude": coords[1],
                "longitude": coords[0],
                "timestamp_str": props.get("timestamp"),  # Keep as raw ISO-8601 string
                "temperature": safe_round(props.get("temperature", {}).get("value")),
                "wind_speed": safe_round(props.get("windSpeed", {}).get("value")),
                "humidity": safe_round(props.get("relativeHumidity", {}).get("value")),
            }
        )

    df = spark.createDataFrame(records, schema=schema)
    # Convert ISO-8601 string to TimestampType, and drop the original
    df = df.withColumn("timestamp", to_timestamp(col("timestamp_str")))
    df = df.drop("timestamp_str")
    # Filter for only the last 7 days
    df = df.filter(col("timestamp") >= date_sub(current_timestamp(), 7))

    return df

INFO:py4j.clientserver:Received command c on object id p0


In [0]:
def upsert_to_table(dataframe: DataFrame, table_name: str) -> None:
    """
    Upsert data into a Unity Catalog table using MERGE.
    
    Args:
        dataframe (DataFrame): The DataFrame to upsert.
        table_name (str): The table name.
    """
    dataframe.createOrReplaceTempView("temp_observations")
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            station_id STRING,
            station_name STRING,
            timezone STRING,
            latitude DOUBLE,
            longitude DOUBLE,
            timestamp TIMESTAMP,
            temperature DOUBLE,
            wind_speed DOUBLE,
            humidity DOUBLE
        )
        USING delta
    """)
    spark.sql(f"""
        MERGE INTO {table_name} target
        USING temp_observations source
        ON target.station_id = source.station_id AND target.timestamp = source.timestamp
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

INFO:py4j.clientserver:Received command c on object id p0


## SQL Queries
The pipeline includes two SQL queries to compute the required metrics:
1. Average Observed Temperature for Last Week (Mon-Sun):
This query calculates the average temperature for the previous week using the date_trunc function to ensure strict Mon-Sun boundaries.


2. Maximum Wind Speed Change:
This query uses a LAG window function to compute differences between consecutive observations and finds the maximum change.

In [0]:
def get_average_temperature_last_week(full_table_name: str) -> None:
    """
    Calculate the average observed temperature for the last week (Mon-Sun).
    
    Args:
        full_table_name (str): The fully qualified table name (e.g., "catalog.schema.table").
        
    Returns:
        None: Displays the results directly.
    """
    query = f"""
        SELECT 
            AVG(temperature) AS avg_temperature
        FROM {full_table_name}
        WHERE 
            timestamp >= date_trunc('week', current_date() - interval 1 week) 
            AND timestamp < date_trunc('week', current_date())
    """
    logger.info("Executing query for average observed temperature last week...")
    result_temperature = spark.sql(query).collect()[0][0]
    logger.info(f"Average observed temperature for last week(Mon-Sun) is: {result_temperature}")

In [0]:
def get_max_wind_speed_change_last_7_days(full_table_name: str) -> None:
    """
    Calculate the maximum wind speed change between two consecutive observations 
    in the last 7 days.
    
    Args:
        full_table_name (str): The fully qualified table name (e.g., "catalog.schema.table").
        
    Returns:
        None: Displays the results directly.
    """
    query = f"""
        WITH ranked_data AS (
            SELECT
                *,
                LAG(wind_speed) OVER (PARTITION BY station_id ORDER BY timestamp) AS prev_wind_speed
            FROM {full_table_name}
            WHERE timestamp >= date_sub(current_date(), 7)
        )
        SELECT 
            MAX(ABS(wind_speed - prev_wind_speed)) AS max_wind_speed_change
        FROM ranked_data
        WHERE prev_wind_speed IS NOT NULL
    """
    logger.info("Executing query for maximum wind speed change last 7 days...")
    result_wind = spark.sql(query).collect()[0][0]
    logger.info(f"Maximum wind speed change last 7 days is: {result_wind}")


INFO:py4j.clientserver:Received command c on object id p0


# Main Function

The main function orchestrates the pipeline by:

1. Fetching weather data for the last 7 days.
2. Transforming the data into a structured format.
3. Upserting the transformed data into the Delta table.
4. Executing the SQL queries to compute the required metrics.

In [0]:
if __name__ == "__main__":
    metadata = fetch_station_metadata(STATION_ID)
    if metadata:
        observations = fetch_weather_data(STATION_ID)
        if observations:
            df = transform_data_to_dataframe(observations, metadata)
            upsert_to_table(df, FULL_TABLE_NAME)
            logger.info("Pipeline executed successfully.")
            get_average_temperature_last_week(FULL_TABLE_NAME)
            get_max_wind_speed_change_last_7_days(FULL_TABLE_NAME)
        else:
            logger.warning("No observations found.")
    else:
        logger.warning("Failed to retrieve station metadata.")

INFO:py4j.clientserver:Received command c on object id p0
INFO:WeatherDataPipeline:Pipeline executed successfully.
INFO:WeatherDataPipeline:Executing query for average observed temperature last week...
INFO:WeatherDataPipeline:Average observed temperature for last week(Mon-Sun) is: 13.309327731092434
INFO:WeatherDataPipeline:Executing query for maximum wind speed change last 7 days...
INFO:WeatherDataPipeline:Maximum wind speed change last 7 days is: 17.709999999999997
